In [ ]:
library(tidyverse)
library(repr)
library(rvest)
library(stringr)
library(janitor)
options(repr.matrix.max.rows = 6)

In [ ]:
players_url <- "https://drive.google.com/uc?export=download&id=1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz"
sessions_url <- "https://drive.google.com/uc?export=download&id=14O91N5OlVkvdGxXNJUj5jIsV5RexhzbB"
players <- read_csv(players_url)
players
sessions <- read_csv(sessions_url)
sessions

In [ ]:
sessions_wrangled <- separate(sessions, start_time, c("start_day", "start_month", "start_year", "start_hour", "start_min"))|>
    separate(end_time, c("end_day", "end_month", "end_year", "end_hour", "end_min"))|>
    mutate(start_day = as.numeric(start_day),
           start_month = as.numeric(start_month),
           start_year = as.numeric(start_year),
           start_hour = as.numeric(start_hour),
           start_min = as.numeric(start_min),
           end_day = as.numeric(end_day),
           end_month = as.numeric(end_month),
           end_year = as.numeric(end_year),
           end_hour = as.numeric(end_hour),
           end_min = as.numeric(end_min))

sessions_wrangled